In [2]:
# ============================================================
# CÉLULA 1 — Leitura do arquivo RAW
# ============================================================
import numpy as np
import os

DATA_PATH = "/Users/edmundobrown/Documents/MLGeral/AI-HealthCare/HREstimation/data/raw/mhealth24_data_public.npy"

if not os.path.exists(DATA_PATH):
    raise FileNotFoundError(f"Arquivo não encontrado: {DATA_PATH}")

print(f"📥 Carregando arquivo RAW: {DATA_PATH}")
data = np.load(DATA_PATH, allow_pickle=True).item()

print("✅ Base carregada com sucesso!")
print("📌 Fases detectadas:", list(data.keys()))

📥 Carregando arquivo RAW: /Users/edmundobrown/Documents/MLGeral/AI-HealthCare/HREstimation/data/raw/mhealth24_data_public.npy
✅ Base carregada com sucesso!
📌 Fases detectadas: ['phase 0', 'phase 1', 'phase 2', 'phase 3', 'phase 4', 'phase 5']


In [5]:
# ============================================================
# NOTEBOOK: Assessment — Auditoria Estrutural (CORRIGIDO)
# ============================================================

import numpy as np
import pandas as pd
import os

# ------------------------------------------------------------
# CONFIGURAÇÕES
# ------------------------------------------------------------
FS = 128               # Hz
HR_WINDOW_S = 30.0     # segundos
SAVE_CSV = True
CSV_OUT = "assessment/eda_structural.csv"
DATA_PATH = "/Users/edmundobrown/Documents/MLGeral/AI-HealthCare/HREstimation/data/raw/mhealth24_data_public.npy"

os.makedirs("assessment", exist_ok=True)

# ------------------------------------------------------------
# FUNÇÕES AUXILIARES
# ------------------------------------------------------------
def find_key(d, contains):
    if isinstance(contains, str):
        contains = [contains]
    for k in d.keys():
        kl = k.lower().strip()
        if all(t in kl for t in contains):
            return k
    return None


def find_all(d, contains):
    if isinstance(contains, str):
        contains = [contains]
    return [k for k in d.keys() if all(t in k.lower().strip() for t in contains)]


def arr_stats(name, arr):
    """Estatísticas robustas com NaN-aware percentis."""
    arr = np.asarray(arr)
    n = arr.size
    arr_f = arr.astype(float)

    quant = np.nanpercentile(arr_f, [0, 1, 25, 50, 75, 99, 100])

    return {
        f"{name}_len": n,
        f"{name}_dtype": str(arr.dtype),
        f"{name}_min": quant[0],
        f"{name}_p01": quant[1],
        f"{name}_p25": quant[2],
        f"{name}_p50": quant[3],
        f"{name}_p75": quant[4],
        f"{name}_p99": quant[5],
        f"{name}_max": quant[6],
        f"{name}_mean": np.nanmean(arr_f),
        f"{name}_std": np.nanstd(arr_f),
        f"{name}_nan_count": np.isnan(arr_f).sum(),
        f"{name}_nan_ratio": np.isnan(arr_f).mean(),
    }


def check_lengths_equal(**arrays):
    lens = {k: (v.size if v is not None else None) for k, v in arrays.items()}
    values = [l for l in lens.values() if l is not None]
    return (len(set(values)) == 1), lens


def imu_energy(x, y, z):
    mag = np.sqrt(x**2 + y**2 + z**2)
    return float(np.nanmean(mag))


# ============================================================
# 1) CARREGAR O ARQUIVO BRUTO (data_raw)
# ============================================================
if not os.path.exists(DATA_PATH):
    raise FileNotFoundError(f"Arquivo não encontrado: {DATA_PATH}")

print(f"📥 Carregando arquivo RAW: {DATA_PATH}")
data_raw = np.load(DATA_PATH, allow_pickle=True).item()
print("✅ Base carregada com sucesso!")
print("📌 Fases detectadas:", list(data_raw.keys()))


# ============================================================
# 2) AUDITORIA ESTRUTURAL (com base SOMENTE no data_raw)
# ============================================================
print("\n===================== AUDITORIA ESTRUTURAL =====================")

rows = []

for phase_name, d in data_raw.items():

    print("\n" + "="*70)
    print(f"🔍 ANALISANDO {phase_name.upper()}")
    print("Chaves encontradas:", list(d.keys()))

    # Detectar chaves de sensores
    ppg_key = find_key(d, "ppg")
    imu_keys = sorted(find_all(d, "imu"))
    hr_key = find_key(d, ["ground", "hr"])

    ppg = d.get(ppg_key)
    imu_x = d.get(imu_keys[0]) if len(imu_keys) >= 1 else None
    imu_y = d.get(imu_keys[1]) if len(imu_keys) >= 2 else None
    imu_z = d.get(imu_keys[2]) if len(imu_keys) >= 3 else None
    hr = d.get(hr_key)

    ok_len, lens = check_lengths_equal(ppg=ppg, imu_x=imu_x, imu_y=imu_y, imu_z=imu_z)

    duration_sec = ppg.size / FS
    expected_hr_windows = int(round(duration_sec / HR_WINDOW_S))
    n_hr = hr.size if isinstance(hr, np.ndarray) else 0
    hr_ok = abs(n_hr - expected_hr_windows) <= 1

    stats = {
        "phase": phase_name,
        "ppg_key": ppg_key,
        "imu_keys": ", ".join(imu_keys),
        "hr_key": hr_key,
        "duration_sec": duration_sec,
        "expected_hr_windows": expected_hr_windows,
        "has_hr": hr is not None,
        "n_hr": n_hr,
        "hr_windows_match": hr_ok,
        "len_consistent": ok_len,
        "len_detail": str(lens),
    }

    # Estatísticas PPG
    if ppg is not None:
        stats.update(arr_stats("ppg", ppg))

    # Estatísticas IMU
    if imu_x is not None:
        stats.update(arr_stats("acc_x", imu_x))
        stats.update(arr_stats("acc_y", imu_y))
        stats.update(arr_stats("acc_z", imu_z))
        stats["acc_energy"] = imu_energy(imu_x, imu_y, imu_z)

    # Estatísticas HR
    if hr is not None:
        stats.update(arr_stats("hr", hr))

    # Windowing integrity
    window_size = int(FS * HR_WINDOW_S)
    samples = ppg.size
    stats["window_size"] = window_size
    stats["expected_windows_signal"] = samples // window_size
    stats["remainder_signal"] = samples % window_size
    stats["windowing_ok"] = (samples % window_size == 0)

    rows.append(stats)

    print(f"• Duração: {duration_sec:.1f}s | HR esperada: {expected_hr_windows}")
    print(f"• HR presente? {hr is not None} | HR len: {n_hr} | Confere? {hr_ok}")
    print(f"• Tamanhos consistentes PPG/IMU? {ok_len} → {lens}")
    print(f"• Windowing: {stats['expected_windows_signal']} janelas | Resto={stats['remainder_signal']}")


# ============================================================
# 3) RESULTADO FINAL DA AUDITORIA
# ============================================================
df = pd.DataFrame(rows).sort_values("phase").reset_index(drop=True)

print("\n" + "="*70)
print("📊 RESUMO FINAL DA AUDITORIA")
display(df)

if SAVE_CSV:
    df.to_csv(CSV_OUT, index=False)
    print(f"💾 Arquivo salvo em: {CSV_OUT}")


# ============================================================
# 4) INSPEÇÃO REAL DO PPG DO ARQUIVO BRUTO
# ============================================================
print("\n===============================================")
print("📌 INSPEÇÃO REAL DO PPG DO ARQUIVO BRUTO")
print("===============================================\n")

for phase, d in data_raw.items():

    ppg_keys = [k for k in d.keys() if "ppg" in k.lower()]

    if not ppg_keys:
        print(f"{phase}: ❌ Nenhum PPG encontrado")
        continue

    for key in ppg_keys:
        ppg = np.asarray(d[key])
        print(f"\n📍 {phase} — chave: '{key}'")
        print(f"  • shape: {ppg.shape}")
        print(f"  • dtype: {ppg.dtype}")
        print(f"  • min: {ppg.min()}")
        print(f"  • max: {ppg.max()}")
        print(f"  • mean: {ppg.mean()}")
        print(f"  • std: {ppg.std()}")
        print(f"  • flat? {ppg.min() == ppg.max()}")
        print(f"  → primeiros 10 valores: {ppg[:10]}")
        print(f"  → últimos 10 valores:   {ppg[-10:]}")

📥 Carregando arquivo RAW: /Users/edmundobrown/Documents/MLGeral/AI-HealthCare/HREstimation/data/raw/mhealth24_data_public.npy
✅ Base carregada com sucesso!
📌 Fases detectadas: ['phase 0', 'phase 1', 'phase 2', 'phase 3', 'phase 4', 'phase 5']

===================== AUDITORIA ESTRUTURAL =====================

🔍 ANALISANDO PHASE 0
Chaves encontradas: ['ground truth HR', 'PPG wrist', 'IMU X wrist', 'IMU Y wrist', 'IMU Z wrist']
• Duração: 11880.0s | HR esperada: 396
• HR presente? True | HR len: 396 | Confere? True
• Tamanhos consistentes PPG/IMU? True → {'ppg': 1520640, 'imu_x': 1520640, 'imu_y': 1520640, 'imu_z': 1520640}
• Windowing: 396 janelas | Resto=0

🔍 ANALISANDO PHASE 1
Chaves encontradas: ['PPG head', 'IMU X head', 'IMU Y head', 'IMU Z head']
• Duração: 11880.0s | HR esperada: 396
• HR presente? False | HR len: 0 | Confere? False
• Tamanhos consistentes PPG/IMU? True → {'ppg': 1520640, 'imu_x': 1520640, 'imu_y': 1520640, 'imu_z': 1520640}
• Windowing: 396 janelas | Resto=0

🔍 A

/var/folders/jm/gm4439qn3dd087mjd3q87mgw0000gn/T/ipykernel_1972/2721719546.py:71: RuntimeWarning: invalid value encountered in sqrt
  mag = np.sqrt(x**2 + y**2 + z**2)


• Duração: 11880.0s | HR esperada: 396
• HR presente? True | HR len: 396 | Confere? True
• Tamanhos consistentes PPG/IMU? True → {'ppg': 1520640, 'imu_x': 1520640, 'imu_y': 1520640, 'imu_z': 1520640}
• Windowing: 396 janelas | Resto=0

🔍 ANALISANDO PHASE 3
Chaves encontradas: ['PPG wrist', 'IMU X wrist', 'IMU Y wrist', 'IMU Z wrist']
• Duração: 11880.0s | HR esperada: 396
• HR presente? False | HR len: 0 | Confere? False
• Tamanhos consistentes PPG/IMU? True → {'ppg': 1520640, 'imu_x': 1520640, 'imu_y': 1520640, 'imu_z': 1520640}
• Windowing: 396 janelas | Resto=0

🔍 ANALISANDO PHASE 4
Chaves encontradas: ['ground truth HR', 'PPG head', 'IMU X head', 'IMU Y head', 'IMU Z head']
• Duração: 1710.0s | HR esperada: 57
• HR presente? True | HR len: 57 | Confere? True
• Tamanhos consistentes PPG/IMU? True → {'ppg': 218880, 'imu_x': 218880, 'imu_y': 218880, 'imu_z': 218880}
• Windowing: 57 janelas | Resto=0

🔍 ANALISANDO PHASE 5
Chaves encontradas: ['PPG head', 'IMU X head', 'IMU Y head', 'IM

,phase,ppg_key,imu_keys,hr_key,duration_sec,expected_hr_windows,has_hr,n_hr,hr_windows_match,len_consistent,...,hr_p99,hr_max,hr_mean,hr_std,hr_nan_count,hr_nan_ratio,window_size,expected_windows_signal,remainder_signal,windowing_ok
0,phase 0,PPG wrist,"IMU X wrist, IMU Y wrist, IMU Z wrist",ground truth HR,11880.0,396,True,396,True,True,...,109.091103,122.190923,76.711321,9.363060,0.0,0.0,3840,396,0,True
1,phase 1,PPG head,"IMU X head, IMU Y head, IMU Z head",None,11880.0,396,False,0,False,True,...,NaN,NaN,NaN,NaN,NaN,NaN,3840,396,0,True
2,phase 2,PPG head,"IMU X head, IMU Y head, IMU Z head",ground truth HR,11880.0,396,True,396,True,True,...,117.017878,126.740717,83.699280,9.684084,0.0,0.0,3840,396,0,True
3,phase 3,PPG wrist,"IMU X wrist, IMU Y wrist, IMU Z wrist",None,11880.0,396,False,0,False,True,...,NaN,NaN,NaN,NaN,NaN,NaN,3840,396,0,True
4,phase 4,PPG head,"IMU X head, IMU Y head, IMU Z head",ground truth HR,1710.0,57,True,57,True,True,...,173.986973,174.268210,153.451010,12.132161,0.0,0.0,3840,57,0,True
5,phase 5,PPG head,"IMU X head, IMU Y head, IMU Z head",None,1710.0,57,False,0,False,True,...,NaN,NaN,NaN,NaN,NaN,NaN,3840,57,0,True


💾 Arquivo salvo em: assessment/eda_structural.csv

📌 INSPEÇÃO REAL DO PPG DO ARQUIVO BRUTO


📍 phase 0 — chave: 'PPG wrist'
  • shape: (1520640,)
  • dtype: int32
  • min: 68896
  • max: 425393
  • mean: 186564.55591724536
  • std: 29521.75674657334
  • flat? False
  → primeiros 10 valores: [78769 78824 78734 78746 78783 78699 78829 78725 78803 78731]
  → últimos 10 valores:   [201629 201621 201542 201463 201780 201814 201610 201529 201433 201529]

📍 phase 1 — chave: 'PPG head'
  • shape: (1520640,)
  • dtype: int32
  • min: 164229
  • max: 411056
  • mean: 248826.41387310607
  • std: 18644.1891688982
  • flat? False
  → primeiros 10 valores: [248148 248599 248404 248548 248845 248504 248846 248600 248640 248619]
  → últimos 10 valores:   [214144 214370 214377 214571 214587 214850 214549 214514 214665 214902]

📍 phase 2 — chave: 'PPG head'
  • shape: (1520640,)
  • dtype: int32
  • min: 144466
  • max: 339767
  • mean: 209096.55530763362
  • std: 27083.015837823415
  • flat? False
  → 

In [4]:
import numpy as np

data_raw = np.load(DATA_PATH, allow_pickle=True).item()

print("Fases detectadas:", list(data_raw.keys()))
print("\n==================== PPG — INSPEÇÃO COMPLETA ====================\n")

for phase, d in data_raw.items():
    # localizar qualquer chave contendo "ppg"
    ppg_keys = [k for k in d.keys() if "ppg" in k.lower()]
    
    if not ppg_keys:
        print(f"{phase}: ❌ Nenhum PPG encontrado")
        continue
    
    for key in ppg_keys:
        ppg = np.asarray(d[key])
        print(f"\n📍 {phase} — chave: '{key}'")
        print(f"  • shape: {ppg.shape}")
        print(f"  • dtype: {ppg.dtype}")

        # estatísticas principais
        print(f"  • min: {ppg.min()}")
        print(f"  • max: {ppg.max()}")
        print(f"  • mean: {ppg.mean()}")
        print(f"  • std: {ppg.std()}")
        
        # checar “flatness”
        is_flat = (ppg.min() == ppg.max())
        print(f"  • flat (min == max)? {is_flat}")

        # mostrar conteúdo
        print("  • primeiros 10 valores:", ppg[:10])
        print("  • últimos 10 valores:  ", ppg[-10:])

Fases detectadas: ['phase 0', 'phase 1', 'phase 2', 'phase 3', 'phase 4', 'phase 5']

==================== PPG — INSPEÇÃO COMPLETA ====================


📍 phase 0 — chave: 'PPG wrist'
  • shape: (1520640,)
  • dtype: int32
  • min: 68896
  • max: 425393
  • mean: 186564.55591724536
  • std: 29521.75674657334
  • flat (min == max)? False
  • primeiros 10 valores: [78769 78824 78734 78746 78783 78699 78829 78725 78803 78731]
  • últimos 10 valores:   [201629 201621 201542 201463 201780 201814 201610 201529 201433 201529]

📍 phase 1 — chave: 'PPG head'
  • shape: (1520640,)
  • dtype: int32
  • min: 164229
  • max: 411056
  • mean: 248826.41387310607
  • std: 18644.1891688982
  • flat (min == max)? False
  • primeiros 10 valores: [248148 248599 248404 248548 248845 248504 248846 248600 248640 248619]
  • últimos 10 valores:   [214144 214370 214377 214571 214587 214850 214549 214514 214665 214902]

📍 phase 2 — chave: 'PPG head'
  • shape: (1520640,)
  • dtype: int32
  • min: 144466
  • ma